In [1]:
import os 
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.chdir("../..")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


# === 4. Load and preprocess data ===
def load_dataset(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['labels'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'labels']]
    return Dataset.from_pandas(df)

train_dataset = load_dataset(paths.english_data_dir / "train_en.tsv")
val_dataset   = load_dataset(paths.english_data_dir / "dev_en.tsv")
test_dataset  = load_dataset(paths.english_data_dir / "dev_test_en.tsv")



# === 5. Tokenization ===
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# === 6. Load model and add LoRA ===
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"]
)

model = get_peft_model(model, lora_config).to(device)

# === 7. Define metrics ===
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# === 9. Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

print("Training complete")

print("Evaluating on test set")
# === 11. Evaluate on test set ===
test_results = trainer.evaluate(eval_dataset=test_dataset)
test_results

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Recall,Precision
1,0.494700,0.669466,0.590909,0.551468,0.283333,0.800000
2,0.604500,0.556190,0.720779,0.717859,0.595833,0.817143
3,0.285800,0.590769,0.731602,0.728158,0.595833,0.841176
4,0.289300,0.532931,0.770563,0.770352,0.712500,0.822115
5,0.539700,0.533729,0.770563,0.770546,0.750000,0.796460
6,0.319100,0.575651,0.774892,0.774380,0.700000,0.840000
7,0.330100,0.678092,0.744589,0.740207,0.591667,0.876543
8,0.290900,0.668493,0.761905,0.759521,0.637500,0.869318
9,0.107500,0.654152,0.770563,0.769000,0.662500,0.864130
10,0.199600,0.676017,0.764069,0.762008,0.645833,0.865922


Training complete
Evaluating on test set


{'eval_loss': 0.620328426361084,
 'eval_accuracy': 0.8016528925619835,
 'eval_f1_macro': 0.7018786572220511,
 'eval_recall': 0.4426229508196721,
 'eval_precision': 0.6585365853658537,
 'eval_runtime': 2.9591,
 'eval_samples_per_second': 163.562,
 'eval_steps_per_second': 40.891,
 'epoch': 15.0}

In [10]:
test_results =trainer.predict(test_dataset)
test_results.metrics

{'test_runtime': 3.1051,
 'test_samples_per_second': 155.871,
 'test_steps_per_second': 38.968}

In [2]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.chdir("../..")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# Set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load and preprocess data
def load_dataset(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['labels'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'labels']]
    return Dataset.from_pandas(df)

train_dataset = load_dataset(paths.english_data_dir / "train_en.tsv")
val_dataset = load_dataset(paths.english_data_dir / "dev_en.tsv")
test_dataset = load_dataset(paths.english_data_dir / "dev_test_en.tsv")

# Tokenization
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Load model and add LoRA
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value"]  # Adjust target modules for XLM-RoBERTa
)

model = get_peft_model(model, lora_config).to(device)

# Define metrics
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

print("Training complete")

# Evaluate on test set
print("Evaluating on test set")
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(test_results)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

d:\A_Facultate\Master\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\flavi\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Recall,Precision
1,0.634500,0.773805,0.480519,0.324561,0.000000,0.000000
2,0.677500,0.643038,0.599567,0.577757,0.358333,0.735043
3,0.502200,0.778800,0.608225,0.576200,0.320833,0.810526
4,0.487900,0.655101,0.707792,0.706105,0.608333,0.780749
5,0.668800,0.608810,0.731602,0.731421,0.729167,0.747863
6,0.481500,0.716128,0.688312,0.683664,0.545833,0.789157
7,0.401400,0.722276,0.694805,0.689390,0.541667,0.807453
8,0.452000,0.750744,0.729437,0.728367,0.641667,0.797927
9,0.440300,0.839097,0.712121,0.704329,0.529167,0.863946
10,0.372200,0.844130,0.712121,0.705158,0.537500,0.854305


d:\A_Facultate\Master\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training complete
Evaluating on test set


{'eval_loss': 0.7191133499145508, 'eval_accuracy': 0.7933884297520661, 'eval_f1_macro': 0.6725747530780679, 'eval_recall': 0.36885245901639346, 'eval_precision': 0.6617647058823529, 'eval_runtime': 5.3618, 'eval_samples_per_second': 90.267, 'eval_steps_per_second': 22.567, 'epoch': 15.0}


In [6]:
import os 
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.chdir("../..")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


# === 4. Load and preprocess data ===
def load_dataset(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['labels'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'labels']]
    return Dataset.from_pandas(df)

train_dataset = load_dataset(paths.english_data_dir / "train_en.tsv")
val_dataset   = load_dataset(paths.english_data_dir / "dev_en.tsv")
test_dataset  = load_dataset(paths.english_data_dir / "dev_test_en.tsv")



# === 5. Tokenization ===
model_name = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# === 6. Load model and add LoRA ===
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query", "key", "value"]
)

model = get_peft_model(model, lora_config).to(device)

# === 7. Define metrics ===
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# === 9. Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

print("Training complete")

print("Evaluating on test set")
# === 11. Evaluate on test set ===
test_results = trainer.evaluate(eval_dataset=test_dataset)
test_results

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Recall,Precision
1,0.642900,0.778054,0.480519,0.324561,0.000000,0.000000
2,0.625900,0.637248,0.621212,0.589183,0.329167,0.849462
3,0.318100,0.760018,0.655844,0.637844,0.416667,0.840336
4,0.479100,0.658077,0.740260,0.737181,0.608333,0.848837
5,0.638800,0.626238,0.770563,0.769717,0.683333,0.845361
6,0.493400,0.629018,0.764069,0.763483,0.687500,0.829146
7,0.317800,0.793180,0.738095,0.734047,0.591667,0.860606
8,0.423700,0.769811,0.753247,0.750988,0.633333,0.853933
9,0.313500,0.821742,0.742424,0.738721,0.600000,0.862275
10,0.158600,0.812664,0.761905,0.759725,0.641667,0.865169


d:\A_Facultate\Master\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training complete
Evaluating on test set


{'eval_loss': 0.8003444075584412,
 'eval_accuracy': 0.8016528925619835,
 'eval_f1_macro': 0.7100928421683139,
 'eval_recall': 0.47540983606557374,
 'eval_precision': 0.6444444444444445,
 'eval_runtime': 5.5425,
 'eval_samples_per_second': 87.325,
 'eval_steps_per_second': 21.831,
 'epoch': 15.0}

In [7]:
import os 
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.chdir("../..")

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


# === 4. Load and preprocess data ===
def load_dataset(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['labels'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'labels']]
    return Dataset.from_pandas(df)

train_dataset = load_dataset(paths.english_data_dir / "train_en.tsv")
val_dataset   = load_dataset(paths.english_data_dir / "dev_en.tsv")
test_dataset  = load_dataset(paths.english_data_dir / "dev_test_en.tsv")



# === 5. Tokenization ===
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# === 6. Load model and add LoRA ===
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"]
)

model = get_peft_model(model, lora_config).to(device)

# === 7. Define metrics ===
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

# === 9. Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

print("Training complete")

print("Evaluating on test set")
# === 11. Evaluate on test set ===
test_results = trainer.evaluate(eval_dataset=test_dataset)
test_results

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

d:\A_Facultate\Master\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\flavi\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Recall,Precision
1,0.615400,0.721338,0.487013,0.364276,0.045833,0.578947
2,0.671000,0.638464,0.582251,0.540673,0.270833,0.783133
3,0.449900,0.623101,0.642857,0.629727,0.437500,0.777778
4,0.409400,0.539783,0.716450,0.716066,0.654167,0.765854
5,0.515600,0.544011,0.733766,0.733615,0.683333,0.777251
6,0.355600,0.578736,0.738095,0.737553,0.666667,0.796020
7,0.322500,0.681708,0.716450,0.713972,0.600000,0.804469
8,0.199700,0.638494,0.744589,0.744282,0.683333,0.796117
9,0.193700,0.716176,0.733766,0.731855,0.625000,0.819672
10,0.191900,0.705840,0.740260,0.739669,0.666667,0.800000


Training complete
Evaluating on test set


{'eval_loss': 0.6768895983695984,
 'eval_accuracy': 0.78099173553719,
 'eval_f1_macro': 0.7095824653563989,
 'eval_recall': 0.5655737704918032,
 'eval_precision': 0.5655737704918032,
 'eval_runtime': 2.9297,
 'eval_samples_per_second': 165.202,
 'eval_steps_per_second': 41.301,
 'epoch': 15.0}